<a href="https://colab.research.google.com/github/nosejohn/eagle-i/blob/main/EAGLE2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# importing necessary datasets
import pandas as pd
eagle2022 = pd.read_csv('eaglei_outages_2022.csv', index_col = False)

In [ ]:
# Convert run_start_time to datetime
eagle2022['run_start_time'] = pd.to_datetime(eagle2022['run_start_time'])

In [ ]:
# Sort the data
eagle2022.sort_values(by=['fips_code', 'run_start_time'], inplace=True)
eagle2022.head()

,fips_code,county,state,sum,run_start_time
25167,1001,Autauga,Alabama,2.0,2022-01-01 12:15:00
25755,1001,Autauga,Alabama,2.0,2022-01-01 12:30:00
30916,1001,Autauga,Alabama,14.0,2022-01-01 14:30:00
31669,1001,Autauga,Alabama,14.0,2022-01-01 14:45:00
32452,1001,Autauga,Alabama,14.0,2022-01-01 15:00:00


In [ ]:
def process_group(group):
    group['run_start_time'] = pd.to_datetime(group['run_start_time'])

    # Calculate time differences between consecutive rows in minutes
    group['time_diff'] = group['run_start_time'].diff().dt.total_seconds() / 60.0

    # Mark rows as new outage based on time difference and sum change
    group['new_outage'] = (group['time_diff'] > 16) | (group['sum'] != group['sum'].shift()) | group['time_diff'].isnull()
    group['outage_id'] = group['fips_code'].astype(str) + '_' + group['new_outage'].cumsum().astype(str)

    # Determine start and end times for each outage
    group['start_time'] = group.groupby('outage_id')['run_start_time'].transform('first')
    group['end_time'] = group.groupby('outage_id')['run_start_time'].transform('last')

    return group[['fips_code', 'county', 'state', 'sum', 'start_time', 'end_time']]


In [ ]:
# Finding whether there is mistyped run_start_time
import re

date_pattern = re.compile(r"\d{4}-\d{2}-\d{2}")

# Find rows with invalid 'run_start_time' formats
invalid_dates = eagle2022[~eagle2022['run_start_time'].astype(str).str.match(date_pattern)]

# Inspect invalid dates to decide on correction
print(invalid_dates)

In [ ]:
# Apply the function to each group
aggregated2022 = eagle2022.groupby(['fips_code', 'county', 'state', 'sum']).apply(process_group).reset_index(drop=True)

print(aggregated2022)

<ipython-input-7-c2cd99445d29>:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  aggregated2022 = eagle2022.groupby(['fips_code', 'county', 'state', 'sum']).apply(process_group).reset_index(drop=True)


        fips_code      county                         state   sum  \
0            1001     Autauga                       Alabama   2.0   
1            1001     Autauga                       Alabama   2.0   
2            1001     Autauga                       Alabama  14.0   
3            1001     Autauga                       Alabama  14.0   
4            1001     Autauga                       Alabama  14.0   
...           ...         ...                           ...   ...   
521225      78030  St. Thomas  United States Virgin Islands   7.0   
521226      78030  St. Thomas  United States Virgin Islands   7.0   
521227      78030  St. Thomas  United States Virgin Islands   1.0   
521228      78030  St. Thomas  United States Virgin Islands  27.0   
521229      78030  St. Thomas  United States Virgin Islands  27.0   

                start_time            end_time  
0      2022-01-01 12:15:00 2022-01-01 12:30:00  
1      2022-01-01 12:15:00 2022-01-01 12:30:00  
2      2022-01-01 14:30:

In [ ]:
aggregated2022['duration'] = aggregated2022['end_time'] - aggregated2022['start_time'] + pd.Timedelta(minutes=15)
aggregated2022['duration'] = aggregated2022['duration'].dt.total_seconds() / 60


In [ ]:
aggregated2022.to_csv('aggregated_eaglei_2022.csv')